**Importing Modules(Libraries)**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml

**Importing the MNIST Dataset** <br>
The dataset is also cached so it can be easily accessed the next time it is needed.


In [2]:
mnist = fetch_openml('mnist_784', version=1)

In [3]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In the dataset we have:
* A DESCR Key describing the data
* A data key containing an array with one row per instance and one column per feature
* A target key containing an array with the labels

**Checking out the arrays**

In [7]:
X, y = mnist["data"], mnist['target']
X.shape

(70000, 784)

In [6]:
y.shape

(70000,)